In [1]:
from __future__ import division
import pandas as pd
import numpy as np 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from xgboost import XGBRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import RidgeCV, LassoCV, Ridge
from sklearn.cross_validation import KFold

In [2]:
def prepare_data():
    train    = pd.read_csv('../input/train.csv')
    test     = pd.read_csv('../input/test.csv')
    labels   = train.Hazard.values
    test_indices = test.ix[:,'Id']
    train.drop('Hazard', axis=1, inplace=True)
    train.drop('Id', axis=1, inplace=True)
    test.drop('Id', axis=1, inplace=True)
    train = np.array(train)
    test = np.array(test)
    for i in range(train.shape[1]):
        if type(train[1,i]) is str:
            lbl = LabelEncoder()
            lbl.fit(list(train[:,i]) + list(test[:,i]))
            train[:,i] = lbl.transform(train[:,i])
            test[:,i] = lbl.transform(test[:,i])
    return train.astype(float), labels, test.astype(float), test_indices

In [3]:
X, y, X_test, test_indices = prepare_data()

Because some of the features in the data are categorical, it is important to do one hot encoding of the data for certain classifiers. Since the features are anonymous, it's possible that all of the features a categorical. For example, one of the features has values of 1, 2, or 3, but those could refer to something categorical (like Region 1, Region 2, and Region 3) or something quantitative (like number of floors in the house).

In this notebook, I'll apply one hot encoding to all of the features since none of them are obviously quantitative.

In [4]:
encoder = OneHotEncoder()
encoder.fit(np.vstack([X, X_test]))
X_oh = encoder.transform(X)
X_test_oh = encoder.transform(X_test)

I will use many different classifiers for the first round of predictions. I will divide them into classifiers that need one hot encoded data and those that do not. You might want to add some code to have each models select the best parameters based on some cross validation, but I'll leave that out here. The n_jobs=-1 parameter will ensure that the models use all available processors. 

For the Ridge and Lasso models, I pass in a cv parameter that tells the model to take the training data and split it up into folds for cross validation. This will help the model choose the best parameters. So for RidgeCV and LassoCV, they are taking the 4 folds that are passed in and using those as the training set. Then they take that training set and split it up into 3 folds. With 2 of those folds, they will train the model using different values of the regularization parameter. Once all the training is done, they can see which value of the regularization parameter produced the best model (by checking the predictions on the left out 3rd fold). The process is repeated leaving out a different fold until all of the folds have been left out. The best regularization parameter overall across all the folds is chosen and then the model is trained on the entire training set using that parameter.

In [6]:
clfs = [('rfr', RandomForestRegressor(n_jobs=-1, max_depth=10)),
        ('xtr', ExtraTreesRegressor(n_jobs=-1, max_depth=10)),
        ('xgb', XGBRegressor())]

clfs_oh = [('rlr', RidgeCV(cv=3, alphas=[0.001, 0.01, 0.1, 1, 10])),
           ('llr', LassoCV(cv=3, alphas=[0.001, 0.01, 0.1, 1, 10]))]

For stacking, we will first generate an intermediate set of data consisting of the predictions of the labels for the test set. Then we will train a model to combine these intermediate predictions into a final prediction. 

The tricky part is figuring out how to train the model that uses the intermediate predictions. In order to train, there must be some intermediate data for which the labels are known. We solve this problem in one of two ways:

1. Set aside some of the training data and make sure it isn't used when training the first round of models. After all of those models are trained, we can make intermediate predictions on both the unused portion of the training set and the test set. Unfortunately, we have not used all of the training data when training the first round of models. The next approach does use all of the training data. 

2. Divide the training data into 5 different folds. Train the initial round of models using 4 of the 5 folds and use those trained models to make intermediate predictions for the unused fold. So far, this seems like exactly the same thing we did in the first approach. Here is the difference. Do the same thing all over again holding out a different fold and make intermediate predictions on the left out fold. Continue this until you have ended up training a set of models 5 different times (once for each of the held out folds). Now you have a set of intermediate data that is the same size as the original training set (once you combine all of the intermediate pieces together).

The approach I'll use is the second. One question you might have is how we will generate the intermediate data for the test set. We will be training our suite of models 5 different times. Which of those times will we use to generate the intermediate data for the test set? The answer is that we will make intermediate predictions for all of the test set 5 different times and average the intermediate predictions together so that we end up with a single set of intermediate predictions for the test set.

In what follows, `X` and `X_oh` will refer to the training set, `X_test` and `X_test_oh` will refer to the test set, `y` will refer to the known labels for the training set, and `test_indices` will refer to the IDs of the test set (used for matching my predictions with the actual labels on the Kaggle server).

In [7]:
n_folds = 5
splits = KFold(X.shape[0], n_folds=n_folds, random_state=1783)

n_classifiers = len(clfs) + len(clfs_oh)
X_intermediate = np.zeros((X.shape[0], n_classifiers))
X_test_intermediate = np.zeros((X_test.shape[0], n_classifiers))
# an X_test_temp array to hold the predictions for the test set 
# before averaging the results of training with each differents
# set of folds
X_test_temp = np.zeros((X_test.shape[0], n_folds)) 
for i, (name, model) in enumerate(clfs):
    print 'Training {0} model'.format(name)
    for j, (ix_train, ix_hold_out) in enumerate(splits):
        model.fit(X[ix_train], y[ix_train])
        X_intermediate[ix_hold_out, i] = model.predict(X[ix_hold_out])
        X_test_temp[:, j] = model.predict(X_test)
    X_test_intermediate[:, i] = X_test_temp.mean(axis=1)

offset = len(clfs)
for i, (name, model) in enumerate(clfs_oh):
    print 'Training {0} model'.format(name)
    for j, (ix_train, ix_hold_out) in enumerate(splits):
        model.fit(X_oh[ix_train], y[ix_train])
        X_intermediate[ix_hold_out, i + offset] = model.predict(X_oh[ix_hold_out])
        X_test_temp[:, j] = model.predict(X_test_oh)
    X_test_intermediate[:, i + offset] = X_test_temp.mean(axis=1)

Training rfr model
Training xtr model
Training xgb model
Training rlr model
Training llr model


At this point we should check to see how accurately each of the models performed. In this competition, the performance metric was the Gini Index (which is a measure of the accuracy of the relative order of the predicted values instead of the accuracy of the predictions themselves).

In [8]:
def normalized_gini(y_true, y_pred):
    # check and get number of samples
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    
    # sort rows on prediction column 
    # (from largest to smallest)
    arr = np.array([y_true, y_pred]).transpose()
    true_order = arr[arr[:,0].argsort()][::-1,0]
    pred_order = arr[arr[:,1].argsort()][::-1,0]
    
    # get Lorenz curves
    L_true = np.cumsum(true_order) / np.sum(true_order)
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    L_ones = np.linspace(0, 1, n_samples)
    
    # get Gini coefficients (area between curves)
    G_true = np.sum(L_ones - L_true)
    G_pred = np.sum(L_ones - L_pred) 
    
    # normalize to true Gini coefficient
    return G_pred/G_true

In [9]:
for i, (name, model) in enumerate(clfs + clfs_oh):
    print 'Gini index for {0}: {1}'.format(name, normalized_gini(y, X_intermediate[:,i]))

Gini index for rfr: 0.329337945294
Gini index for xtr: 0.32964940103
Gini index for xgb: 0.368957427389
Gini index for rlr: 0.350814296872
Gini index for llr: 0.351772339372


Now let's check the correlation between the predictions of the various models. 

In [10]:
df = pd.DataFrame(X_intermediate, columns=zip(*(clfs + clfs_oh))[0])
df_test = pd.DataFrame(X_test_intermediate, columns=zip(*(clfs + clfs_oh))[0])
df.corr()

,rfr,xtr,xgb,rlr,llr
rfr,1.000000,0.808450,0.795750,0.709196,0.718194
xtr,0.808450,1.000000,0.832333,0.746259,0.759787
xgb,0.795750,0.832333,1.000000,0.906242,0.914116
rlr,0.709196,0.746259,0.906242,1.000000,0.993958
llr,0.718194,0.759787,0.914116,0.993958,1.000000


Notice how the Ridge and Lasso models have almost perfect correlation. You might want to remove one of them from the ensemble since it is unlikely to offer new information while training. I'll leave all of them in.

Now all of the intermediate data is created and stored in `X_intermediate` and `X_test_intermediate`. All that's left to do is train a new model on this data. Since you might want to check how much stacking has improved the accuracy or continue on with another level of stacking, I'll first get predictions by the same method used above.

In [11]:
combinerModel = RidgeCV(cv=3, alphas=[0.001, 0.003, 0.01, 0.03, 0.1, 0.3, 1, 3, 10, 30, 100, 300, 1000])
X_second_stage = np.zeros((X.shape[0], ))
# use splits and X_test_temp from earlier in the code
for j, (ix_train, ix_hold_out) in enumerate(splits):
    combinerModel.fit(X_intermediate[ix_train], y[ix_train])
    X_second_stage[ix_hold_out] = combinerModel.predict(X_intermediate[ix_hold_out])
    X_test_temp[:, j] = combinerModel.predict(X_test_intermediate)
X_test_second_stage = X_test_temp.mean(axis=1)

In [12]:
print 'Regularization parameter chosen for Ridge model is {0}'.format(combinerModel.alpha_)
print 'Gini index for combined model: {0}'.format(normalized_gini(y, X_second_stage))

Regularization parameter chosen for Ridge model is 30
Gini index for combined model: 0.370934145549


We can see that the stacking has slightly improved the performance of the best model (xgboost) even though the other models were all weaker than xgboost.

At this point, we could train a few more models using the intermediate data and then continue to a second round of stacking, but I'll just finish here and use the discovered regularization parameter above to retrain using all of the intermediate data and then make final predictions. (It's possible that RidgeCV retrains with all of the data by default, but I couldn't find any mention of it in the documentation, so I'll just retrain to be safe.)

In [13]:
finalModel = Ridge()
finalModel.fit(X_intermediate, y)
X_test_predictions = combinerModel.predict(X_test_intermediate)

In [15]:
submission = {}
submission['Id'] = test_indices.astype(int)
submission['Hazard'] = X_test_predictions
df = pd.DataFrame(submission, columns=['Id', 'Hazard'])
df.to_csv('../submissions/Stackingv01s01.csv', index=False)